In [17]:
import datetime
import tarfile
import json
import bz2

In [38]:
#keywords to filter the tweets
keywords=['gmo','gmos','gm food','gm foods','transgenic', 'transgenics','genetically-modified','genetically modified']

# initializing array which will hold the filtered tweets
filteredTweets=[]


# opening the tar file
with tarfile.open('data/May.tar', 'r') as tar:
    
    # iterating through all files and directories
    for member in tar.getmembers():
        
        #skipping directories
        if member.isdir():
            continue
        
        #looking for the target files json.bz2
        if member.name.endswith('.json.bz2'):
            
            # extracting each file
            file = tar.extractfile(member)
         
            #reading in and decompressing the file
            noBz2 = bz2.decompress(file.read()).decode('utf-8')
            
            # handling each tweet to find the match
            for line in noBz2.splitlines():
                
                #each line is one tweet in json format, therefore loading line by line
                tweet = json.loads(line)

                # making sure the tweet has the fields 'text' and 'created_at'. If not they'll be skipped
                if 'text' not in tweet or 'created_at' not in tweet or tweet['user']['lang'] != 'en':
                    continue

                # Skipping retweets
                if tweet['text'].startswith('RT') and tweet.get('retweeted_status') is not None:
                    continue
                
                # storing fields if present
                tweetTxt = tweet['text']
                created_at = tweet['created_at']

                # Searching matches in text
                if any(keyword in tweetTxt.lower().split() for keyword in keywords):                
                    # Add the matching tweet to the list
                    filteredTweets.append({
                        'text': tweetTxt,
                        'created_at': created_at
                    })
                    

#writting the fields from matching tweets to a json output file
with open('tweets.json', 'w') as output:
    json.dump(filteredTweets, output)


In [70]:
from pyspark.sql.types import *

In [80]:
# from pyspark.sql.functions import from_unixtime, unix_timestamp

# customSchema=StructType([
#     StructField('text', StringType(), True),
#     StructField('created_at', StringType(), True)])

# tweet=spark.read.load('hdfs://localhost:9000/user1/tweetsJan.json', format="'json', schema=customSchema)

# tweet=tweet.withColumn('created_at', from_unixtime(unix_timestamp(tweet['created_at'], 'E MMM d HH:mm:ss Z y')).cast(TimestampType()))

# tweet.printSchema()
# tweet.show()


root
 |-- text: string (nullable = true)
 |-- created_at: timestamp (nullable = true)



Py4JJavaError: An error occurred while calling o576.showString.
: org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to recognize 'E MMM d HH:mm:ss Z y' pattern in the DateTimeFormatter. 1) You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0. 2) You can form a valid datetime pattern with the guide from https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToRecognizePatternAfterUpgradeError(QueryExecutionErrors.scala:936)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkLegacyFormatter$1.applyOrElse(DateTimeFormatterHelper.scala:187)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkLegacyFormatter$1.applyOrElse(DateTimeFormatterHelper.scala:180)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalys

In [153]:
from pyspark.sql.functions import udf
import datetime


def parse_date(date_str):
    #return datetime.datetime.strptime(date_str, 'E MMM d HH:mm:ss Z y')
    return datetime.datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y')



parse_date_udf= udf(parse_date, TimestampType())

tweet=spark.read.json('hdfs://localhost:9000/user1/tweetsJan.json')
tweet=tweet.withColumn('created_at', parse_date_udf(tweet['created_at']))

tweet.printSchema()
tweet.show()


root
 |-- created_at: timestamp (nullable = true)
 |-- text: string (nullable = true)

+-------------------+--------------------+
|         created_at|                text|
+-------------------+--------------------+
|2012-01-16 17:56:18|Mouse developed i...|
|2012-01-16 17:41:21|@ScottMadin *shak...|
|2012-01-17 01:48:55|@ur4everREagent @...|
|2012-01-16 22:07:36|Organic_Store fou...|
|2012-01-16 19:29:02|Leaked: US to Sta...|
|2012-01-16 09:04:59|@asdfghjklvivy vi...|
|2012-01-10 17:32:51|Scarey facts abou...|
|2012-01-10 18:12:45|@_HandSomeBait Fr...|
|2012-01-10 18:05:13|Interesting: Top ...|
|2012-01-10 15:55:28|Bourbon of proof:...|
|2012-01-10 16:01:26|Monsanto’s GMO Co...|
|2012-01-10 16:13:00|Monsanto’s GMO Co...|
|2012-01-10 16:24:56|Bourbon of proof:...|
|2012-01-10 09:30:36|ม่ายย!!!! จาเอาตอ...|
|2012-01-11 04:38:38|RT @cinnamonshelp...|
|2012-01-10 12:51:40|Transgenic Crop P...|
|2012-01-10 14:48:32|I think its hard ...|
|2012-02-01 01:24:57|@maureenogle ? RT...|
|2012-01-3